In [2]:
import pandas as pd

In [74]:
connection_string = f"postgres:{pw}@localhost:5432/Austin_Housing_Market"
engine = create_engine(f'postgresql://{connection_string}')

In [75]:
# engine = create_engine("mysql+pymysql://user:pw@host/db", pool_pre_ping=True)

In [76]:
Base = automap_base()

In [77]:
Base.prepare(engine, reflect=True)

annual_sales = Base.classes.annual_sales
monthly_sales = Base.classes.monthly_sales
price_distribution = Base.classes.price_distribution


In [78]:
session = Session(engine)


In [79]:
session.query(price_distribution.year_2011).__dict__

{'session': <sqlalchemy.orm.session.Session at 0x209b7998370>,
 '_propagate_attrs': immutabledict({'compile_state_plugin': 'orm', 'plugin_subject': <Mapper at 0x209b7d2bac0; price_distribution>}),
 '_raw_columns': [Column('year_2011', DOUBLE_PRECISION(precision=53), table=<price_distribution>, nullable=False)]}

In [80]:
session.query(annual_sales).all()

In [9]:
# pull in annual home sales excel file and create dataframe
annual_data_df = pd.read_excel('Resources/annual_home_sales.xlsx')

annual_data_df.head()




,year,sales,dollar_volume,average_price,median_price,total_listings,months_inventory
0,1990,7068,579669083,82013,72252,4412,6.5
1,1991,7485,661674435,88400,75865,3552,4.6
2,1992,8389,832936284,99289,82929,3020,3.6
3,1993,9784,1076687995,110046,90949,2860,3.2
4,1994,10418,1207051547,115862,95158,3645,3.9


In [66]:
annual_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   year              33 non-null     int64  
 1   sales             33 non-null     int64  
 2   dollar_volume     33 non-null     int64  
 3   average_price     33 non-null     int64  
 4   median_price      33 non-null     int64  
 5   total_listings    33 non-null     int64  
 6   months_inventory  33 non-null     float64
dtypes: float64(1), int64(6)
memory usage: 1.9 KB


In [3]:
# pull in monthly home sales excel file and create dataframe
monthly_data_df = pd.read_excel('Resources/monthly_home_sales.xlsx')
monthly_data_df.head()

,date,sales,dollar_volume,average_price,median_price,total_listings,months_inventory
0,1990-01-01,558,46937286,84117,71000,4391,7.9
1,1990-02-01,444,37056240,83460,71199,4343,8.7
2,1990-03-01,431,34722653,80563,72497,4459,9.3
3,1990-04-01,569,34616822,60838,71998,4545,9.1
4,1990-05-01,549,44907102,81798,73694,4731,9.3


In [7]:
# add column for year to monthly data
monthly_data_df['year'] = pd.DatetimeIndex(monthly_data_df['date']).year


monthly_data_df

,date,sales,dollar_volume,average_price,median_price,total_listings,months_inventory,year
0,1990-01-01,558,46937286,84117,71000,4391,7.9,1990
1,1990-02-01,444,37056240,83460,71199,4343,8.7,1990
2,1990-03-01,431,34722653,80563,72497,4459,9.3,1990
3,1990-04-01,569,34616822,60838,71998,4545,9.1,1990
4,1990-05-01,549,44907102,81798,73694,4731,9.3,1990
...,...,...,...,...,...,...,...,...
395,2022-12-01,2510,1385555532,552014,450000,7402,2.6,2022
396,2023-01-01,1632,908398440,556617,449955,7231,2.6,2023
397,2023-02-01,2125,1141406548,537132,435000,7103,2.6,2023
398,2023-03-01,2781,1568805664,564116,450000,8007,3.0,2023


In [15]:
# calculate median home price by year
med_price_by_year = monthly_data_df.groupby('year')['median_price'].mean()
med_price_by_year

year
1990     72189.083333
1991     75673.500000
1992     82550.833333
1993     90484.500000
1994     95316.083333
1995     99823.500000
1996    107990.000000
1997    111424.500000
1998    116871.583333
1999    125828.083333
2000    143075.750000
2001    149129.750000
2002    153321.166667
2003    153188.000000
2004    152381.333333
2005    160007.250000
2006    170859.750000
2007    182718.583333
2008    186294.583333
2009    184423.333333
2010    188714.583333
2011    188156.083333
2012    201181.000000
2013    218700.250000
2014    237864.583333
2015    259841.083333
2016    278240.833333
2017    292805.000000
2018    303968.000000
2019    312615.333333
2020    341533.000000
2021    448537.500000
2022    500114.166667
2023    449988.750000
Name: median_price, dtype: float64

In [20]:
# percent increase from 1990 to 2023
percent_increase_1990 = ((med_price_by_year[2023] - med_price_by_year[1990]) / med_price_by_year[1990]) * 100
percent_increase_1990

# percent increase from 2000 to 2023
percent_increase_2000 = ((med_price_by_year[2023] - med_price_by_year[2000]) / med_price_by_year[2000]) * 100
percent_increase_2000

# percent increase from 2010 to 2023
percent_increase_2010 = ((med_price_by_year[2023] - med_price_by_year[2010]) / med_price_by_year[2010]) * 100
percent_increase_2010

# percent increase from 2020 to 2023
percent_increase_2020 = ((med_price_by_year[2023] - med_price_by_year[2020]) / med_price_by_year[2020]) * 100
percent_increase_2020

# print resutls to 2 decimal places

print(f'Percent increase in median home price from 1990 to 2023: {round(percent_increase_1990,2)}%')
print(f'Percent increase in median home price from 2000 to 2023: {round(percent_increase_2000,2)}%')
print(f'Percent increase in median home price from 2010 to 2023: {round(percent_increase_2010,2)}%')
print(f'Percent increase in median home price from 2020 to 2023: {round(percent_increase_2020,2)}%')


Percent increase in median home price from 1990 to 2023: 523.35%
Percent increase in median home price from 2000 to 2023: 214.51%
Percent increase in median home price from 2010 to 2023: 138.45%
Percent increase in median home price from 2020 to 2023: 31.76%


In [68]:
# pull in price distribution excel file and create dataframe
price_data_df = pd.read_excel('Resources/price_distribution.xlsx')
price_data_df.head()


,price_distribution,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,"$0 - $69,999",4.5,2.9,1.7,0.9,0.6,0.4,0.2,0.1,0.1,0.1,0.0,0.0
1,"$70,000 - $99,999",6.5,5.3,3.3,2.0,1.1,0.8,0.5,0.3,0.2,0.2,0.0,0.0
2,"$100,000 - $149,999",21.6,19.5,16.0,11.0,6.5,3.7,2.5,1.7,1.2,0.8,0.3,0.1
3,"$150,000 - $199,999",21.0,21.4,22.3,22.2,19.5,15.2,11.2,7.9,6.5,3.8,1.3,0.4
4,"$200,000 - $249,999",13.0,14.0,15.2,17.0,18.6,19.6,20.6,20.6,19.9,14.7,3.8,1.1


In [69]:
austin_housing = pd.read_csv('Resources/austinHousingData_2021.csv')

austin_housing.head()

,zpid,city,streetAddress,zipcode,latitude,longitude,propertyTaxRate,garageSpaces,hasAssociation,hasCooling,...,numOfMiddleSchools,numOfHighSchools,avgSchoolDistance,avgSchoolRating,avgSchoolSize,MedianStudentsPerTeacher,numOfBathrooms,numOfBedrooms,numOfStories,homeImage
0,111373431,pflugerville,14424 Lake Victor Dr,78660,30.430632,-97.663078,1.98,2,True,True,...,1,1,1.266667,2.666667,1063,14,3.0,4,2,111373431_ffce26843283d3365c11d81b8e6bdc6f-p_f...
1,120900430,pflugerville,1104 Strickling Dr,78660,30.432672,-97.661697,1.98,2,True,True,...,1,1,1.400000,2.666667,1063,14,2.0,4,1,120900430_8255c127be8dcf0a1a18b7563d987088-p_f...
2,2084491383,pflugerville,1408 Fort Dessau Rd,78660,30.409748,-97.639771,1.98,0,True,True,...,1,1,1.200000,3.000000,1108,14,2.0,3,1,2084491383_a2ad649e1a7a098111dcea084a11c855-p_...
3,120901374,pflugerville,1025 Strickling Dr,78660,30.432112,-97.661659,1.98,2,True,True,...,1,1,1.400000,2.666667,1063,14,2.0,3,1,120901374_b469367a619da85b1f5ceb69b675d88e-p_f...
4,60134862,pflugerville,15005 Donna Jane Loop,78660,30.437368,-97.656860,1.98,0,True,True,...,1,1,1.133333,4.000000,1223,14,3.0,3,2,60134862_b1a48a3df3f111e005bb913873e98ce2-p_f.jpg


In [70]:
austin_housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15171 entries, 0 to 15170
Data columns (total 46 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   zpid                        15171 non-null  int64  
 1   city                        15171 non-null  object 
 2   streetAddress               15171 non-null  object 
 3   zipcode                     15171 non-null  int64  
 4   latitude                    15171 non-null  float64
 5   longitude                   15171 non-null  float64
 6   propertyTaxRate             15171 non-null  float64
 7   garageSpaces                15171 non-null  int64  
 8   hasAssociation              15171 non-null  bool   
 9   hasCooling                  15171 non-null  bool   
 10  hasGarage                   15171 non-null  bool   
 11  hasHeating                  15171 non-null  bool   
 12  hasSpa                      15171 non-null  bool   
 13  hasView                     151